In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('all_tables2.csv')

In [4]:
df 

,result,table,_start,_stop,_time,_value,Time,_field,_measurement,applicationId,...,devAddr,devEui,deviceClassEnabled,deviceName,deviceProfileId,deviceProfileName,gatewayId,nsTime,tenantId,tenantName
0,_result,0,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 12:49:06.067459+00:00,1.0,2024-04-02T12:49:06.048715924+00:00,adr,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T12:49:05.832808333+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
1,_result,1,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 12:49:06.067459+00:00,1.0,2024-04-02T12:49:06.048715924+00:00,confirmed,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T12:49:05.832808333+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
2,_result,2,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 12:49:06.067459+00:00,18.0,2024-04-02T12:49:06.048715924+00:00,data_size,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T12:49:05.832808333+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
3,_result,3,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 12:49:06.067459+00:00,5.0,2024-04-02T12:49:06.048715924+00:00,dr,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T12:49:05.832808333+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
4,_result,4,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 12:49:06.067459+00:00,33425.0,2024-04-02T12:49:06.048715924+00:00,fCnt,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T12:49:05.832808333+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,_result,1877,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 13:29:01.774269+00:00,7.2,2024-04-02T13:29:01.535+00:00,rxInfo_snr,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T13:29:01.551941703+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
1878,_result,1878,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 13:29:01.774269+00:00,29669.0,2024-04-02T13:29:01.535+00:00,rxInfo_uplinkId,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T13:29:01.551941703+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
1879,_result,1879,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 13:29:01.774269+00:00,868500000.0,2024-04-02T13:29:01.535+00:00,txInfo_frequency,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T13:29:01.551941703+00:00,52f14cd4-c6f1-4fbd-8f87-4025e1d49242,ChirpStack
1880,_result,1880,2024-04-02 11:29:06.211761+00:00,2024-04-02 13:29:06.211761+00:00,2024-04-02 13:29:01.774269+00:00,125000.0,2024-04-02T13:29:01.535+00:00,txInfo_modulation_bandwidth,Uplink_data,9debbb59-d4d6-4306-8141-f25029222bca,...,00e58d09,0080e11500073358,CLASS_A,new_new_device,fa31de69-bdb7-4b51-b47f-0bd978be1603,susee_dev,7076ff005607214b,2024-04-02T13:29:01.551941703+00:00,52f14cd4-c6f1-4fbd-8f87-40

### Data merging
- The queried table from the influxDB is organized in such a way that the _field and their corresponding values are stored in seperate rows.
- But these are the outputs of the same JSON mqtt messages comming from the chirpstack. 
- Thus, it is desired to convert the _field rows into columns that stores the corresponding values based on the timestamps.
- We chose the _time column as the timestamp for the merger
- This is done in 3 steps
    - 1st a pivoted dataframe is generated converting the _field rows into columns 
    - Next the _field and _value columns are dropped and an intermediate DF is created
    - Finally, the 2 dataframes are merged with reference to the _time column. This merges the duplicate entries in the intermediate DF 

In [40]:
df_pivoted = df.pivot(index='_time', columns='_field', values='_value').reset_index()

In [41]:
df_droped = df.drop(columns=['_field', '_value'],axis=1)

In [45]:
df_mergeed = df_pivoted.merge(df_droped, on='_time')

print(df_mergeed)

                                 _time  adr  confirmed  data_size   dr  \
0     2024-04-02 12:49:06.067459+00:00  1.0        1.0       18.0  5.0   
1     2024-04-02 12:49:06.067459+00:00  1.0        1.0       18.0  5.0   
2     2024-04-02 12:49:06.067459+00:00  1.0        1.0       18.0  5.0   
3     2024-04-02 12:49:06.067459+00:00  1.0        1.0       18.0  5.0   
4     2024-04-02 12:49:06.067459+00:00  1.0        1.0       18.0  5.0   
...                                ...  ...        ...        ...  ...   
1877  2024-04-02 13:29:01.774269+00:00  1.0        1.0        7.0  5.0   
1878  2024-04-02 13:29:01.774269+00:00  1.0        1.0        7.0  5.0   
1879  2024-04-02 13:29:01.774269+00:00  1.0        1.0        7.0  5.0   
1880  2024-04-02 13:29:01.774269+00:00  1.0        1.0        7.0  5.0   
1881  2024-04-02 13:29:01.774269+00:00  1.0        1.0        7.0  5.0   

         fCnt  fPort  rxInfo_board  rxInfo_channel  rxInfo_location_altitude  \
0     33425.0   29.0           

In [46]:
df_mergeed.to_csv('merged_data.csv', index=0)